## QAOA

QAOAはVQEと同様で特定のansatzを組み合わせ最適化問題に使います。   
量子断熱発展を使うことで始点から終点まで基底状態を保つことができます。

現時刻をt、全体のスケジュールをTとし、初期のハミルトニアンを $H_{start}$、最後のハミルトニアンを $H_{final}$ とします。

$$
H_{temp} = (1-\frac{t}{T})H_{start} + \frac{t}{T}H_{final}
$$


$T\rightarrow\infty$ のとき、基底状態は以下のような固有状態になります。

$$
H_{temp}\mid \psi \rangle = E_{0temp}\mid \psi \rangle
$$

ここで時間発展は以下のようになります。

$$
\mid \psi_{t+1} \rangle = U \mid \psi_t \rangle = e^{-iHt}  \mid \psi_t \rangle
$$

回路については

![https___qiita-user-contents.imgix.net_https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2Fe10f1843-cc16-cdfe-e4a6-e2fbaab6df9f.png_ixlib=rb-1.2.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/218694/f1de7c18-1b0f-8212-b30d-208c0d88e94c.png)

1. 状態の設定
2. qaoa ansatz

の2つの部分にわかれています。

また、qaoa ansatz はさらに2つの部分にわかれていて、

1. cost ハミルトニアン
2. mixer ハミルトニアン

上の例では、Hゲートが設定の部分でその他が ansatz 部分です。

ansatz に関しては、CX-Rz-CX-Rz が cost ハミルトニアン で Rx が mixer ハミルトニアンとなります。
cost ハミルトニアンは細かく述べると, CX-Rz-CX の部分が重みの部分で Rz が バイアスの部分です.

ここで標準状態として |+> を準備し、 HゲートとXゲート(時間発展させるためRxに変換させます。)を mixer として使います。 

簡単な問題をしてみます。

In [14]:
!pip install blueqat obaq

In [7]:
from blueqat import vqe
from blueqat.pauli import qubo_bit as q

h = 2*q(1)-4*q(0)*q(1)
step = 2

result = vqe.Vqe(vqe.QaoaAnsatz(h, step)).run()
print(result.most_common(12))

(((1, 1), 0.975929633353178), ((1, 0), 0.015300121678013385), ((0, 0), 0.007080945393067879), ((0, 1), 0.0016892995757399336))


求める解のビットの組み合わせと確率振幅から確率が得られます。

## 基本の量子アニーリング

Obaqというツールの中にQaoaQaoaAnsatzというものを入れてみました。QaoaQaoaAnsatzは初期状態では量子アニーリングに設定されています。mixerにXというものを使い、初期状態に|+>を設定しています。
mixerはXを使います。Xは量子ビットの0と1を反転させる操作に対応します。

また、初期状態の|+>は全ての量子ビットにアダマールゲートHをかけることで実現します。今回定式化はblueqat機能を使って、QUBO形式でやってみます。

In [22]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.pauli import qubo_bit as q
from blueqat.vqe import Vqe
from obaq.ansatz import *

h = 2*q(1)-4*q(0)*q(1)
step = 2

result = Vqe(QaoaQaoaAnsatz(h,step)).run()
print(result.most_common(12))

(((1, 1), 0.9862018661764347), ((0, 0), 0.010840899546257964), ((0, 1), 0.0024879737729535134), ((1, 0), 0.00046926050435334776))


練習のために一応同じ設定をしてみると、

In [23]:
h = -q(0) - q(0)*q(1)

#mixerの準備
mixer = X[0]+X[1]
initial_state = Circuit(2).h[0,1]

runner = Vqe(QaoaQaoaAnsatz(h.to_expr().simplify(), 2, initial_state, mixer))
result = runner.run()
print(result.most_common(12))

(((1, 1), 0.769567560217707), ((0, 0), 0.19853901331586488), ((1, 0), 0.027124354625384886), ((0, 1), 0.004769071841041183))


２量子ビット使い、mixerで量子ビットの両方にXを指定します。
また、初期状態にはアダマールをかけて準備します。
同じような答えが出るはずです。

## mixerを変えてみる
mixerを変えてみましょう。mixerを変えることで量子ビットの探索を変えることができます。これによって、従来のXでの量子ビットの反転以外の操作もすることができます。おさらいにXゲートを見てみると、

$$
X\mid 0 \rangle = 
\begin{bmatrix}
0&1\\
1&0
\end{bmatrix}
\begin{bmatrix}
1\\
0
\end{bmatrix}
=
\begin{bmatrix}
0\\
1
\end{bmatrix}
$$

のように、量子ビットのベクトル[1,0]を[0,1]に反転させることができます。量子断熱過程では、上記のXハミルトニアンを時間発展演算子$e^{-iXt}$を使って時間発展しています。時間発展させることで、直感的にはかき混ぜ効果の強さを制御できます。

次にmixerを(XX+YY)/2という俗にいうXYmixerにしてみます。これは01と10を交換するゲートに対応します。こちらは２量子ビットゲートなので、例えば、片方の量子ビットを0に、もう片方を1にします。

$$
q_0 = 
\begin{bmatrix}
1\\
0
\end{bmatrix}
,q_1 = 
\begin{bmatrix}
0\\
1
\end{bmatrix}
$$

次にXYmixerの使い方を見てみます。XYmixerは(XX+YY)/2というゲート操作に対応します。テンソル積を使って表現ができます。

$$
\begin{bmatrix}
a&b \\
c&d
\end{bmatrix}
\otimes
\begin{bmatrix}
e&f \\
g&h
\end{bmatrix} \\
=
\begin{bmatrix}
a*
\begin{bmatrix}
e&f \\
g&h
\end{bmatrix}
&b*
\begin{bmatrix}
e&f \\
g&h
\end{bmatrix}
\\
c*
\begin{bmatrix}
e&f \\
g&h
\end{bmatrix}
&d*
\begin{bmatrix}
e&f \\
g&h
\end{bmatrix}
\end{bmatrix}
=
\begin{bmatrix}
ae&af&be&bf\\
ag&ah&bg&bh\\
ce&cf&de&df\\
cg&ch&dg&dh
\end{bmatrix}
$$

ちなみに基本的にハミルトニアンに利用できるのは、パウリ演算子です。

$$
X=
\begin{bmatrix}
0&1 \\
1&0
\end{bmatrix},
Y=
\begin{bmatrix}
0&-i \\
i&0
\end{bmatrix},
Z=
\begin{bmatrix}
1&0 \\
0&-1
\end{bmatrix}
$$

これらを組み合わせてやります。やってみると、

$$
X_0X_1 = 
\begin{bmatrix}
0&1\\
1&0
\end{bmatrix}
\otimes
\begin{bmatrix}
0&1\\
1&0
\end{bmatrix}
=
\begin{bmatrix}
0&0&0&1\\
0&0&1&0\\
0&1&0&0\\
1&0&0&0
\end{bmatrix}
$$

また、

$$
Y_0Y_1 = 
\begin{bmatrix}
0&-i\\
i&0
\end{bmatrix}
\otimes
\begin{bmatrix}
0&-i\\
i&0
\end{bmatrix}
=
\begin{bmatrix}
0&0&0&-1\\
0&0&1&0\\
0&1&0&0\\
-1&0&0&0
\end{bmatrix}
$$

ということで、上記を足し合わせて2で割ると、

$$
(X_0X_1 + Y_0Y_1)/2 
=
\begin{bmatrix}
0&0&0&0\\
0&0&1&0\\
0&1&0&0\\
0&0&0&0
\end{bmatrix}
$$

となりました。4*4の行列は、|00>,|01>,|10>,|11>の量子状態のベクトル要素を新しい量子状態のベクトル要素に変換しますが、ここでは特に|01>が|10>に、|10>が|01>に変換される操作になっています。

これを使ってみます。ここで大事なのが、mixerに対応した量子状態を準備する必要があります。上記の行列に対応する自明な量子状態の一つとして、[0,1,1,0]があります。これは、

```python
Circuit().h[0].cx[0,1].x[0]
```

として量子もつれを使って作ることができます。

In [25]:
h = -q(0) - q(0)*q(1)

#mixerの準備
XYmixer = 0.5*X[0]*X[1] + 0.5*Y[0]*Y[1]
XYinitial_state = Circuit().h[0].cx[0,1].x[0]

runner = Vqe(QaoaQaoaAnsatz(h.to_expr().simplify(), 2, XYinitial_state, XYmixer))
result = runner.run()
print(result.most_common(12))

(((1, 0), 0.9999979423375291), ((0, 1), 2.057662469107359e-06), ((0, 0), 3.8515701477840927e-32), ((1, 1), 7.703719777548941e-34))


となりました。本来は(1,1)が出て欲しいところですが、01と10で制約をかけてしまいましたので、コスト関数としてのハミルトニアンの結果が反映されていません。これは正しい操作で、いかにmixerと初期状態の制約が強いかというのを確認できます。問題設定のハミルトニアンをちょっと変えてみて、

In [26]:
h = q(0)+q(1)

#mixerの準備
XYmixer = 0.5*X[0]*X[1] + 0.5*Y[0]*Y[1]
XYinitial_state = Circuit().h[0].cx[0,1].x[0]

runner = Vqe(QaoaQaoaAnsatz(h.to_expr().simplify(), 2, XYinitial_state, XYmixer))
result = runner.run()
print(result.most_common(12))

(((1, 0), 0.49999999999999883), ((0, 1), 0.49999999999999883), ((0, 0), 7.703719777548942e-32), ((1, 1), 7.703719777548942e-32))


こんなふうにすれば正しく答えも出ます。

## (XX-YY)/2も
XX-YYをやってみると、

$$
(X_0X_1 - Y_0Y_1)/2 = 
\begin{bmatrix}
0&0&0&1\\
0&0&0&0\\
0&0&0&0\\
1&0&0&0
\end{bmatrix}
$$

と、なり|00>と|11>の交換になります。下記のように|00>と|11>のもつれを作ると、これに対応した制約がかかります。初期状態とmixerを調整して、

In [27]:
h = q(0)+q(1)

#mixerの準備
XY2mixer = 0.5*X[0]*X[1] - 0.5*Y[0]*Y[1]
XY2initial_state = Circuit().h[0].cx[0,1]

runner = Vqe(QaoaQaoaAnsatz(h.to_expr().simplify(), 2, XY2initial_state, XY2mixer))
result = runner.run()
print(result.most_common(12))

(((0, 0), 0.9999999999999984), ((1, 1), 5.05765025770774e-19), ((0, 1), 2.234078735489194e-32), ((1, 0), 5.007417848383253e-33))


こちらは、問題ハミルトニアンのコスト関数の条件と、制約条件を両方満たしたものがきれいに出ています。

## QAOAを用いた交通最適化
もう一つの例としてフォルクスワーゲン社の交通流最適を参考にしてみましょう。

「Quantum Computing at Volkswagen:
Traffic Flow Optimization using the D-Wave Quantum Annealer」   
引用：https://www.dwavesys.com/sites/default/files/VW.pdf

ステップは以下のようになります、

1. 各車における経路をそれぞれ2つ決める。 (古典)
2. 混雑量を計算する。 (古典)
3. 1で決めた各車のルートを変えて混雑量を最適化させる。 (QAOA)

まずスタート地点をA、ゴール地点をBとして、各経路に0から11まで番号を振っています。
各車それぞれ2つずつ経路を考え、混雑量を最適化させることを目標とします。

![0_jdvbeWCvUMG-UKis.jpeg](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/218694/d401be2a-5971-ca4f-c00e-63403372bae8.jpeg)

```python
car1  
route1-1 (q0)：s0,s3,s6,s9  
route1–2 (q1)：s0,s3,s8,s11  

car2  
route2–1 (q2):s0,s3,s8,s11  
route2–2 (q3):s2,s7,s10,s11
```

## Mixerを選ぶ
提案した2つの経路から1つだけ選ぶので、片方は1でもう片方は0となります。
これらの経路を量子ビットで考えるなら $q_0$ と $q_1$ のどちらかは 1 で もう１つは 0 となります。 $q_2$ と $q_3$ も同様に考えます。MixerはXYmixerを選びます。

初期状態は量子もつれを用います。

In [12]:
from blueqat import Circuit
Circuit().h[0].cx[0,1].x[0].m[:].run(shots=100)

Counter({'01': 47, '10': 53})

この回路は |01> と |10> のもつれを作ります。これを使うことで探索空間を制限させることができます。この量子もつれを各量子ビットに用いることで、以下のように2つもつれを作ることができます。

In [13]:
from blueqat import Circuit
Circuit().h[0].cx[0,1].x[0].h[2].cx[2,3].x[2].m[:].run(shots=100)

Counter({'0110': 31, '1010': 22, '0101': 19, '1001': 28})

## cost 関数の混雑量
混雑量を計算してみましょう。経路を以下のように考えます。

car1  
route1-1(q0)：s0,s3,s6,s9  
route1–2 (q1)：s0,s3,s8,s11  

car2  
route2–1(q2):s0,s3,s8,s11  
route2–2(q3):s2,s7,s10,s11  

考えた経路が何回現れたか見ることで、混雑量を計算できます。

s0 から s11 までの経路から、全ての経路の混雑量を2乗して計算することができます。

$$
h=(q_0+q_1+q_2)^2+(q_0+q_1+q_2)^2+q_0^2+q_0^2+(q_1+q_2)^2+(q_1+q_2+q_3)^2+q_3^2+q_3^2+q_3^2\\
=4q_0+4q_1+4q_2+4q_3+4q_0q_1+4q_0q_2+8q_1q_2+2q_1q_3+2q_2q_3
$$

ここで, $0^2=0,1^2=1$ を用いた。

これが cost ハミルトニアン です。これをかけることで、問題を解くことができます。

In [29]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.pauli import qubo_bit as q
from blueqat.vqe import AnsatzBase, Vqe
from obaq.ansatz import *

#mixerの準備
XYinit = Circuit().h[0].cx[0,1].x[0].h[2].cx[2,3].x[2]
XYmixer = 0.5*X[0]*X[1] + 0.5*Y[0]*Y[1] + 0.5*X[2]*X[3] + 0.5*Y[2]*Y[3]

h = 4*q(0)+4*q(1)+4*q(2)+4*q(3)+4*q(0)*q(1)+4*q(0)*q(2)+8*q(1)*q(2)+2*q(1)*q(3)+2*q(2)*q(3)

runner = Vqe(QaoaQaoaAnsatz(h, 2, XYinit, XYmixer))
result = runner.run()

print(result.most_common(12))

(((1, 0, 0, 1), 0.70698278558037), ((1, 0, 1, 0), 0.21683907324924093), ((0, 1, 0, 1), 0.04656599490568942), ((0, 1, 1, 0), 0.029612146264695514), ((1, 0, 1, 1), 2.0029671421627247e-32), ((1, 0, 0, 0), 1.0014835710813626e-32), ((0, 0, 0, 1), 9.039251682133586e-33), ((1, 1, 1, 0), 7.592095925388722e-33), ((0, 1, 1, 1), 5.248159098455218e-33), ((1, 1, 0, 1), 3.9634837409346916e-33), ((0, 1, 0, 0), 3.274080905458301e-33), ((0, 0, 1, 0), 3.14225521611568e-33))


1001という正解のルートが出ました。